In [1]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [2]:
# change this
curr_quarter = 'Jun 2022'
prev_quarter = 'Mar 2022'
len(stock_ids)

58

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 

#stock_name = 'tcs'

    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"
    
    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')
        
        test_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")
            
        return test_list
    
    test_list = get_list(actual_link)
        
    print(len(test_list))
    n = len(test_list) 
    
    if n < 114:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        print(len(test_list))
    
    q = 0
        
    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   
        
    else:
        p = 6

    df_qreults = pd.DataFrame()
    
    if 'Gross NPA %'  in test_list:
        endkey = 'Gross NPA %'
    else:
        endkey = 'Raw PDF'

    if curr_quarter in test_list:
        quarter = curr_quarter
    else:
        quarter = prev_quarter

    #df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-p:test_list.index('Sep 2021')+1]
    #df_qreults['EPS(Q)'] = test_list[test_list.index('Raw PDF')-p-q-1:test_list.index('Raw PDF')-q]
    s = test_list.index(endkey)  - test_list.index('EPS in Rs')-2
    df_qreults['Quarter'] = test_list[test_list.index(quarter)-s:test_list.index(quarter)+1]    
    df_qreults['EPS(Q)'] = test_list[test_list.index('EPS in Rs')+1:test_list.index(endkey)]

    try:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Expenses\xa0+')-s-1:test_list.index('Expenses\xa0+')]
    except:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Interest')-s-1:test_list.index('Interest')] 
    
    
    df_qreults['Sales(Q)']  = df_qreults['Sales(Q)'].str.replace(",", "")

    df_qreults['EPS(Q)'] = df_qreults['EPS(Q)'].astype(float).fillna(0.0)
    df_qreults['Sales(Q)'] = df_qreults['Sales(Q)'].astype(float).fillna(0.0)

    from pandas.tseries.offsets import MonthEnd
    import datetime

    df_qreults['Quarter'] = pd.to_datetime(df_qreults['Quarter'], format="%b %Y") + MonthEnd(1) + datetime.timedelta(days=46)
    df_qreults

    df_qreults['EPS(TTM)'] = df_qreults['EPS(Q)'] + df_qreults['EPS(Q)'].shift(1) + df_qreults['EPS(Q)'].shift(2) + \
                                df_qreults['EPS(Q)'].shift(3)
    
    df_qreults['Sales(TTM)'] = df_qreults['Sales(Q)'] + df_qreults['Sales(Q)'].shift(1) + df_qreults['Sales(Q)'].shift(2) + \
                                df_qreults['Sales(Q)'].shift(3)

    import pandas_datareader.data as web
    import yfinance as yfin
    yfin.pdr_override()

    df_stock = pd.DataFrame()

    for date in df_qreults['Quarter']:
        print(date)

        end_date = date
        start_date = (date + datetime.timedelta(days=-7)).strftime('%Y-%m-%d')


        stock_id = stock_name.upper() + '.NS'
        tmp = pd.DataFrame(web.DataReader(tickers = stock_id, data_source='yahoo', start = start_date, end = end_date))
        tmp = tmp.tail(1)
        tmp['Stock'] = stock_id
        tmp['Quarter'] = end_date  
        tmp['Close'] = round(tmp['Close'],2)
        tmp.reset_index(inplace = True)
        cols = ['Stock','Quarter','Close']
        tmp =tmp[cols]    
        df_stock = df_stock.append(tmp)

    df_stock_analysis = df_stock.merge(df_qreults, on='Quarter').tail(6)
    df_stock_analysis['PE(TTM)'] = round(df_stock_analysis['Close']/df_stock_analysis['EPS(TTM)'],2)
    
    df_analysis = df_analysis.append(df_stock_analysis)
    
df_analysis

ASIANPAINT
172
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-0

[*********************100%***********************]  1 of 1 completed
COLPAL
16
172
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[******

[*********************100%***********************]  1 of 1 completed
GODREJCP
174
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[*******

[*********************100%***********************]  1 of 1 completed
HINDUNILVR
174
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[*****

[*********************100%***********************]  1 of 1 completed
ITC
174
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[************

[*********************100%***********************]  1 of 1 completed
MCDOWELL-N
172
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[*****

[*********************100%***********************]  1 of 1 completed
RELIANCE
174
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[*******

[*********************100%***********************]  1 of 1 completed
TATAPOWER
174
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[******

[*********************100%***********************]  1 of 1 completed
WIPRO
174
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[**********

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM)
7,ASIANPAINT.NS,2021-05-16,2774.50,8.88,6651.0,32.73,21712.0,84.77
8,ASIANPAINT.NS,2021-08-15,2988.70,5.93,5585.0,36.38,24374.0,82.15
9,ASIANPAINT.NS,2021-11-15,3122.90,6.21,7096.0,33.93,26120.0,92.04
10,ASIANPAINT.NS,2022-02-15,3143.45,10.59,8527.0,31.61,27859.0,99.44
11,ASIANPAINT.NS,2022-05-16,3064.00,8.87,7893.0,31.60,29101.0,96.96
...,...,...,...,...,...,...,...,...
8,ZEEL.NS,2021-08-15,183.25,2.23,1775.0,10.24,8193.0,17.90
9,ZEEL.NS,2021-11-15,312.70,2.81,1979.0,12.07,8449.0,25.91
10,ZEEL.NS,2022-02-15,249.35,3.11,2113.0,11.02,7833.0,22.63
11,ZEEL.NS,2022-05-16,239.25,1.89,2323.0,10.04,8190.0,23.83


In [4]:
df_analysis_final = pd.DataFrame()
for stock_id in df_analysis['Stock'].unique():
    tmp = df_analysis[df_analysis['Stock'] == stock_id]
    tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
    tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
    tmp['Prev_Sales'] = tmp['Sales(Q)'].shift(1)
    tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['FR%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS(TTM)'])*100/tmp['Prev_EPS(TTM)'],2)
    tmp['SR%'] = round(tmp['EPS(TTM)']*(tmp['PE(TTM)'] - tmp['Prev_PE'])*100/tmp['Prev_Close'],2)
    tmp['TR%'] = tmp['FR%'] + tmp['SR%']
    tmp['Sales%'] = round((tmp['Sales(Q)'] - tmp['Prev_Sales'])*100/tmp['Prev_Sales'],2)
    tmp['EPS%'] = round((tmp['EPS(Q)'] - tmp['Prev_EPS(Q)'])*100/tmp['Prev_EPS(Q)'],2)
    tmp['Return%'] = tmp['FR%'] + tmp['SR%']
    df_analysis_final = df_analysis_final.append(tmp)
    
df_analysis_final

<ipython-input-4-e22d75f1c612>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
<ipython-input-4-e22d75f1c612>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
<ipython-input-4-e22d75f1c612>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
7,ASIANPAINT.NS,2021-05-16,2774.50,8.88,6651.0,32.73,21712.0,84.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ASIANPAINT.NS,2021-08-15,2988.70,5.93,5585.0,36.38,24374.0,82.15,8.88,32.73,6651.0,84.77,2774.50,11.15,-3.44,7.71,-16.03,-33.22,7.71
9,ASIANPAINT.NS,2021-11-15,3122.90,6.21,7096.0,33.93,26120.0,92.04,5.93,36.38,5585.0,82.15,2988.70,-6.73,11.23,4.50,27.05,4.72,4.50
10,ASIANPAINT.NS,2022-02-15,3143.45,10.59,8527.0,31.61,27859.0,99.44,6.21,33.93,7096.0,92.04,3122.90,-6.84,7.49,0.65,20.17,70.53,0.65
11,ASIANPAINT.NS,2022-05-16,3064.00,8.87,7893.0,31.60,29101.0,96.96,10.59,31.61,8527.0,99.44,3143.45,-0.03,-2.49,-2.52,-7.44,-16.24,-2.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,ZEEL.NS,2021-08-15,183.25,2.23,1775.0,10.24,8193.0,17.90,2.87,8.33,1966.0,23.00,191.60,22.93,-27.26,-4.33,-9.72,-22.30,-4.33
9,ZEEL.NS,2021-11-15,312.70,2.81,1979.0,12.07,8449.0,25.91,2.23,10.24,1775.0,17.90,183.25,17.87,52.76,70.63,11.49,26.01,70.63
10,ZEEL.NS,2022-02-15,249.35,3.11,2113.0,11.02,7833.0,22.63,2.81,12.07,1979.0,25.91,312.70,-8.70,-11.56,-20.26,6.77,10.68,-20.26
11,ZEEL.NS,2022-05-16,239.25,1.89,2323.0,10.04,8190.0,23.83,3.11,11.02,2113.0,22.63,249.35,-8.89,4.83,-4.06,9.94,-39.23,-4.06


In [13]:
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
df_growth = df_analysis_final[cols]
df_growth[df_growth['Quarter'] == '2022-08-15'].sort_values(by='Sales%', ascending=False).head(20)

,Stock,Quarter,Sales%,EPS%,Return%
12,INDIGO.NS,2022-08-15,60.27,-36.72,23.99
12,BERGEPAINT.NS,2022-08-15,26.14,14.98,3.05
12,PIDILITIND.NS,2022-08-15,23.69,39.20,23.59
12,TATAPOWER.NS,2022-08-15,21.20,58.60,4.29
12,TITAN.NS,2022-08-15,21.13,54.01,18.16
12,NAUKRI.NS,2022-08-15,15.64,-71.60,27.99
12,DMART.NS,2022-08-15,14.25,50.68,34.23
12,SUNPHARMA.NS,2022-08-15,13.92,-190.52,3.50
12,HEROMOTOCO.NS,2022-08-15,12.69,-3.43,12.75
12,DABUR.NS,2022-08-15,12.07,50.00,16.13


In [6]:
df_analysis_final.to_csv('data/Portfolio_FvS_QoQ_Analysis.csv', index=False)

In [7]:
df_growth = pd.read_csv('data/Portfolio_FvS_QoQ_Analysis.csv')
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
#df_growth = df_growth[cols]
df_growth = df_growth.dropna()
df_growth

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
1,ASIANPAINT.NS,2021-08-15,2988.70,5.93,5585.0,36.38,24374.0,82.15,8.88,32.73,6651.0,84.77,2774.50,11.15,-3.44,7.71,-16.03,-33.22,7.71
2,ASIANPAINT.NS,2021-11-15,3122.90,6.21,7096.0,33.93,26120.0,92.04,5.93,36.38,5585.0,82.15,2988.70,-6.73,11.23,4.50,27.05,4.72,4.50
3,ASIANPAINT.NS,2022-02-15,3143.45,10.59,8527.0,31.61,27859.0,99.44,6.21,33.93,7096.0,92.04,3122.90,-6.84,7.49,0.65,20.17,70.53,0.65
4,ASIANPAINT.NS,2022-05-16,3064.00,8.87,7893.0,31.60,29101.0,96.96,10.59,31.61,8527.0,99.44,3143.45,-0.03,-2.49,-2.52,-7.44,-16.24,-2.52
5,ASIANPAINT.NS,2022-08-15,3427.85,10.60,8607.0,36.27,32123.0,94.51,8.87,31.60,7893.0,96.96,3064.00,14.78,-2.90,11.88,9.05,19.50,11.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,ZEEL.NS,2021-08-15,183.25,2.23,1775.0,10.24,8193.0,17.90,2.87,8.33,1966.0,23.00,191.60,22.93,-27.26,-4.33,-9.72,-22.30,-4.33
344,ZEEL.NS,2021-11-15,312.70,2.81,1979.0,12.07,8449.0,25.91,2.23,10.24,1775.0,17.90,183.25,17.87,52.76,70.63,11.49,26.01,70.63
345,ZEEL.NS,2022-02-15,249.35,3.11,2113.0,11.02,7833.0,22.63,2.81,12.07,1979.0,25.91,312.70,-8.70,-11.56,-20.26,6.77,10.68,-20.26
346,ZEEL.NS,2022-05-16,239.25,1.89,2323.0,10.04,8190.0,23.83,3.11,11.02,2113.0,22.63,249.35,-8.89,4.83,-4.06,9.94,-39.23,-4.06


In [8]:
df_growth[(df_growth['Sales%'] > 5) & (df_growth['EPS%'] > 5)]['Stock'].value_counts()[0:20]

MINDTREE.NS      4
BERGEPAINT.NS    3
LT.NS            3
EICHERMOT.NS     3
DMART.NS         3
JUBLFOOD.NS      3
DABUR.NS         3
RELIANCE.NS      3
NATIONALUM.NS    3
TITAN.NS         3
DIVISLAB.NS      2
M&M.NS           2
ASIANPAINT.NS    2
BHARTIARTL.NS    2
PIDILITIND.NS    2
TATASTEEL.NS     2
ZEEL.NS          2
LTI.NS           2
ULTRACEMCO.NS    2
TATAPOWER.NS     2
Name: Stock, dtype: int64

In [9]:
df_tmp_final = pd.DataFrame()
for stock in df_growth['Stock'].unique():
    df_tmp = df_growth[df_growth['Stock'] == stock][cols]
    df_tmp['sales_flag'] = 0
    df_tmp['eps_flag'] = 0
    df_tmp.loc[df_tmp['Sales%'] < 0, 'sales_flag'] = 1
    df_tmp.loc[df_tmp['EPS%'] < 0, 'eps_flag'] = 1
    df_tmp['sales_weight'] =  df_tmp['sales_flag']*5 + df_tmp['sales_flag'].shift(1)*4 + df_tmp['sales_flag'].shift(2)*3+df_tmp['sales_flag'].shift(3)*2+df_tmp['sales_flag'].shift(4)*1
    df_tmp['eps_weight'] =  df_tmp['eps_flag']*5 + df_tmp['eps_flag'].shift(1)*4 + df_tmp['eps_flag'].shift(2)*3+df_tmp['eps_flag'].shift(3)*2+df_tmp['eps_flag'].shift(4)*1
    df_tmp['total_weight'] = df_tmp['sales_weight'] + df_tmp['eps_weight'] 
    df_tmp = df_tmp.tail(1)
    df_tmp_final = df_tmp_final.append(df_tmp)

In [10]:
df_tmp_final[(df_tmp_final['Sales%'] > 5) & (df_tmp_final['EPS%'] > 5)].sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
299,TATAPOWER.NS,2022-08-15,21.20,58.60,4.29,0,0,3.0,1.0,4.0
257,RELIANCE.NS,2022-08-15,5.75,10.81,8.48,0,0,1.0,5.0,6.0
53,DABUR.NS,2022-08-15,12.07,50.00,16.13,0,0,4.0,4.0,8.0
5,ASIANPAINT.NS,2022-08-15,9.05,19.50,11.88,0,0,5.0,5.0,10.0
23,BERGEPAINT.NS,2022-08-15,26.14,14.98,3.05,0,0,5.0,5.0,10.0
71,DMART.NS,2022-08-15,14.25,50.68,34.23,0,0,5.0,5.0,10.0
179,JUBLFOOD.NS,2022-08-15,6.72,16.33,20.79,0,0,5.0,5.0,10.0
329,TITAN.NS,2022-08-15,21.13,54.01,18.16,0,0,5.0,5.0,10.0
251,PIDILITIND.NS,2022-08-15,23.69,39.20,23.59,0,0,5.0,8.0,13.0


In [11]:
df_tmp_final.sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
17,BAJFINANCE.NS,2022-08-15,3.20,7.31,32.51,0,0,0.0,1.0,1.0
131,HINDUNILVR.NS,2022-08-15,6.23,3.26,18.27,0,0,1.0,1.0,2.0
83,EICHERMOT.NS,2022-08-15,6.39,0.09,31.96,0,0,1.0,1.0,2.0
299,TATAPOWER.NS,2022-08-15,21.20,58.60,4.29,0,0,3.0,1.0,4.0
101,HCLTECH.NS,2022-08-15,3.84,-8.61,-9.07,0,1,0.0,5.0,5.0
221,MINDTREE.NS,2022-08-15,7.73,-0.31,20.28,0,1,0.0,5.0,5.0
197,LTI.NS,2022-08-15,5.14,-0.58,19.45,0,1,0.0,6.0,6.0
203,M&M.NS,2022-08-15,9.56,-1.89,41.58,0,1,1.0,5.0,6.0
143,ICICIBANK.NS,2022-08-15,4.28,-4.41,29.08,0,1,0.0,6.0,6.0
173,ITC.NS,2022-08-15,11.68,4.71,19.33,0,0,5.0,1.0,6.0


In [14]:
scrip = 'MOTILALOFS.NS'
df_growth[df_growth['Stock'] == scrip][cols]

,Stock,Quarter,Sales%,EPS%,Return%
7,MOTILALOFS.NS,2021-05-16,NaN,NaN,NaN
8,MOTILALOFS.NS,2021-08-15,-21.38,-52.01,16.14
9,MOTILALOFS.NS,2021-11-15,47.49,142.36,17.74
10,MOTILALOFS.NS,2022-02-15,-22.52,-55.37,-7.16
11,MOTILALOFS.NS,2022-05-16,2.54,23.39,-9.03
12,MOTILALOFS.NS,2022-08-15,-28.35,-89.55,-6.05
